# Getting data from svt.se
The svt.se site does not have a search function and the articles are thus scraped through searching on google

### Packages and Connector

In [21]:
import pandas as pd
import numpy as np
import requests, re, time, os
import random
from tqdm import tqdm   # bar that shows progress of scraping
from bs4 import BeautifulSoup

In [23]:
class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          self.log.flush()
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
          self.log.flush()
    else:
      t = time.time()
      ratelimit()
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
      self.log.flush()
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return None,call_id

In [18]:
from datetime import timedelta, date
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)
#Define the interval for possible scraping articles 
start_dt = date(2018, 8, 2)
end_dt = date(2018, 8, 6)
dates=[]
for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%b %d, %Y"))

In [20]:
dates

['Aug 02, 2018',
 'Aug 03, 2018',
 'Aug 04, 2018',
 'Aug 05, 2018',
 'Aug 06, 2018']

In [25]:
def ratelimit():
    "this function handles the rate of our calls."
    time.sleep(0.5)

Define connection to the internet

In [41]:
connector = Connector('svt_scrape.csv')
url = 'https://www.google.com/async/bgasy?ei=pVZeXfiPJOqOrwSxgK7IAw&yv=3&async=_fmt:jspb'
response, call_id = connector.get(url, 'svt_call')

<Response [200]>

In [43]:
! pip install fake_useragent

  Running setup.py bdist_wheel for fake-useragent: started
  Running setup.py bdist_wheel for fake-useragent: finished with status 'done'
  Stored in directory: C:\Users\dukic\AppData\Local\pip\Cache\wheels\5e\63\09\d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [57]:
import urllib
from fake_useragent import UserAgent

query = "'site:svt.se flyktingar'"     # Key search words
query = urllib.parse.quote_plus(query) # Format into URL encoding
number_result = 100

ua = UserAgent()

google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number_result)
response = requests.get(google_url, {"User-Agent": ua.random})
soup = BeautifulSoup(response.text, "html.parser")

result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})

links = []
titles = []
descriptions = []
for r in result_div:
    # Checks if each element is present, else, raise exception
    try:
        link = r.find('a', href = True)
        title = r.find('div', attrs={'class':'vvjwJb'}).get_text()
        description = r.find('div', attrs={'class':'s3v9rd'}).get_text()
        
        # Check to make sure everything is present before appending
        if link != '' and title != '' and description != '': 
            links.append(link['href'])
            titles.append(title)
            descriptions.append(description)
    # Next loop if one element is not present
    except:
        continue

In [58]:
links

['/url?q=https://www.svt.se/nyheter/utrikes/ny-fn-rapport-rekordmanga-flyktingar-i-varlden&sa=U&ved=2ahUKEwjZn43LmpbkAhUkpIsKHd6SBoIQFjAGegQIBRAB&usg=AOvVaw2f1DFC2m49idhvB5z-9Gkf',
 '/url?q=https://www.svt.se/nyheter/utrikes/migration-och-flyktingar-dominerar-eu-toppmotet&sa=U&ved=2ahUKEwjZn43LmpbkAhUkpIsKHd6SBoIQFjAHegQIYhAB&usg=AOvVaw3PlY1_x7RbgXe0I7NXs398',
 '/url?q=https://www.svt.se/nyheter/utrikes/danmark-vill-skicka-flyktingar-som-begar-brott-till-ode-o&sa=U&ved=2ahUKEwjZn43LmpbkAhUkpIsKHd6SBoIQFjAIegQIIxAB&usg=AOvVaw2SfXXtDg0PJpmwpljfPaUb',
 '/url?q=https://www.svt.se/nyheter/utrikes/venezuelas-kris-jamfors-med-flyktingkrisen-pa-medelhavet&sa=U&ved=2ahUKEwjZn43LmpbkAhUkpIsKHd6SBoIQFjAJegQIYBAB&usg=AOvVaw39uU62z7OiSU955NLTtdL4',
 '/url?q=https://www.svt.se/nyheter/utrikes/spanien-tar-emot-flyktingarna-italien-nekar&sa=U&ved=2ahUKEwjZn43LmpbkAhUkpIsKHd6SBoIQFjAKegQIXxAB&usg=AOvVaw0RKqlvxcBhMfP8cWatULDq',
 '/url?q=https://www.svt.se/nyheter/lokalt/vasterbotten/hur-fordelas-flyktin